# Setting up inputs for the gridded DBPM run
**Author**: Denisse Fierro Arcos  
**Date**: 2025-01-10  
  
In this notebook, we will prepare all necessary inputs need to run the gridded version of DBPM. Note that this step needs to be completed only once for each model run.  

## Loading relevant libraries

In [2]:
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')
import xarray as xr
import json
import pandas as pd
import numpy as np
import useful_functions as uf
from dask.distributed import Client

## Start a cluster for parallelisation

In [2]:
client = Client(threads_per_worker = 1)

## Defining output folder

In [3]:
base_folder = '/g/data/vf71/la6889/dbpm_inputs/east_antarctica/'
out_folder = os.path.join(base_folder, 'gridded_params')
os.makedirs(out_folder, exist_ok = True)

## Transforming DBPM parameters to Python-friendly format
These parameters are the outputs of the `sizeparam` function for R. They were calculated and stored in the script ([04_calculating_dbpm_fishing_params](new_workflow/04_calculating_dbpm_fishing_params.R)) as a `json` file.

In [6]:
#Loading gridded parameters
gridded_params = json.load(open('gridded_params_testing.json'))

#Transforming them to Python friendly format
gridded_python = uf.gridded_param_python(gridded_params)

### Adding useful entries to gridded parameters
These variables are used multiple times within the `gridded_size_model` script, so they will be added for convenience.

In [7]:
# Size classes
log10_size_bins = np.array(gridded_params['log10_size_bins'])
# Minimum (log10) size class for predators
log10_ind_min_pred_size = log10_size_bins[gridded_python['ind_min_pred_size']]
log10_ind_min_detritivore_size = log10_size_bins[gridded_python['ind_min_detritivore_size']]
log10_ind_min_fish_pred = log10_size_bins[gridded_python['ind_min_fish_pred']]
log10_ind_min_fish_det = log10_size_bins[gridded_python['ind_min_fish_det']]

#Adding new variables to gridded parameters
gridded_python['log10_ind_min_pred_size'] = log10_ind_min_pred_size
gridded_python['log10_ind_min_detritivore_size'] = log10_ind_min_detritivore_size

### Saving gridded parameters

In [6]:
out_gridded = os.path.join(out_folder,
                           'dbpm_size_params_fao-58_python.json')
#Save to disk
with open(out_gridded, 'w') as outfile:
    json.dump(gridded_python, outfile)

## Loading gridded input data
Gridded data are needed both as direct input to DBPM and to calculate params, such as habitat preference, pelagic predator and benthic detritivore size spectrum, among other things.

In [8]:
depth = xr.open_zarr(
    os.path.join(base_folder, 'gridded', 
                 'gfdl-mom6-cobalt2_obsclim_deptho_15arcmin_fao-58_fixed.zarr/'))
depth = depth['deptho'].sel(lat = slice(-60, None), lon = slice(70, 80))

In [9]:
int_phy_zoo = xr.open_zarr(os.path.join(
    base_folder, 'gridded', 
    'gfdl-mom6-cobalt2_obsclim_intercept_15arcmin_fao-58_monthly_1961_2010.zarr/'))
int_phy_zoo = (int_phy_zoo['intercept'].sel(time = '1961').
                sel(lat = slice(-60, None), lon = slice(70, 80)))

In [10]:
slope_phy_zoo_mat = xr.open_zarr(os.path.join(
    base_folder, 'gridded', 
    'gfdl-mom6-cobalt2_obsclim_slope_15arcmin_fao-58_monthly_1961_2010.zarr/'))
slope_phy_zoo_mat = (slope_phy_zoo_mat['slope'].sel(time = '1961').
                sel(lat = slice(-60, None), lon = slice(70, 80)))

In [11]:
sea_surf_temp = xr.open_zarr(os.path.join(
    base_folder, 'gridded', 
    'gfdl-mom6-cobalt2_obsclim_tos_15arcmin_fao-58_monthly_1961_2010.zarr/'))
sea_surf_temp = (sea_surf_temp['tos'].sel(time = '1961').
                sel(lat = slice(-60, None), lon = slice(70, 80)))

In [12]:
sea_floor_temp = xr.open_zarr(os.path.join(
    base_folder, 'gridded', 
    'gfdl-mom6-cobalt2_obsclim_tob_15arcmin_fao-58_monthly_1961_2010.zarr/'))
sea_floor_temp = (sea_floor_temp['tob'].sel(time = '1961').
       sel(lat = slice(-60, None), lon = slice(70, 80)))

### Creating time variable that contains a timestep for initialisation of DBPM

In [13]:
#Adding new 'time_init' input that includes timestep for initialisation of DBPM
time_init = np.array(
    pd.date_range((int_phy_zoo.indexes['time'].to_datetimeindex().min()-
                   pd.DateOffset(months = 1)),
                  int_phy_zoo.indexes['time'].to_datetimeindex().max(),
                  freq = 'MS'), dtype = 'datetime64[ns]')

/jobfs/132146532.gadi-pbs/ipykernel_3873301/1065116443.py:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  pd.date_range((int_phy_zoo.indexes['time'].to_datetimeindex().min()-
/jobfs/132146532.gadi-pbs/ipykernel_3873301/1065116443.py:5: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  int_phy_zoo.indexes['time'].to_datetimeindex().max(),


## Creating gridded variables needed for DBPM run
We will create data array versions of the gridded inputs.

### DBPM size classes

In [14]:
# Log10 size of individuals found in the model
log10_size_bins_mat = xr.DataArray(data = log10_size_bins,
                                   dims = ['size_class'], 
                                   coords = {'size_class': log10_size_bins})

# size of individuals found in the model
size_bin_vals = 10**log10_size_bins_mat

### Fishing effort

In [16]:
effort = (xr.DataArray(gridded_python['effort'], dims = 'time', 
                       coords = {'time': int_phy_zoo.time}).
          expand_dims({'lat': depth.lat, 'lon': depth.lon}).
          transpose('time', 'lat', 'lon').
          chunk({'time': '50MB', 'lat': '50MB', 'lon': '50MB'}))

### Setting habitat preferences
Predator coupling to benthic prey, which is depth dependent, 0.75 above 500 m, 0.5 between 500 and 1800 m, and 0 below 1800 m. Predator coupling to pelagic prey is equal to `1-(benthic coupling)`.  
  
These values were suggested by Clive Trueman based on stable isotope work, and proportion of biomass. Rockall Trough studies.

In [18]:
pref_benthos = (0.8*np.exp((-1/250*depth))).chunk({'lat': '50MB', 'lon': '50MB'})
pref_pelagic = (1-pref_benthos)

### Initiliasing values for pelagic predators and benthic detritivores

In [20]:
init_pred = (xr.DataArray(data = gridded_python['init_pred'], 
                          dims = ['size_class'], 
                          coords = {'size_class': log10_size_bins}).
             expand_dims({'lat': depth.lat, 'lon': depth.lon}).
             chunk({'lat': '50MB', 'lon': '50MB', 'size_class': '50MB'}))

init_detritivores = (xr.DataArray(data = gridded_python['init_detritivores'], 
                                  dims = ['size_class'], 
                                  coords = {'size_class': log10_size_bins}).
                     expand_dims({'lat': depth.lat, 'lon': depth.lon}).
                     chunk({'lat': '50MB', 'lon': '50MB', 'size_class': '50MB'}))

### Calculating constant growth and mortality

In [21]:
pred_prey_mat = uf.pred_prey_matrix(log10_size_bins)

constant_growth = xr.DataArray(uf.gphi_f(pred_prey_mat, 
                                         gridded_python['log10_pred_prey_ratio'],
                                         gridded_python['log_prey_pref']),
                               dims = ['size_class', 'sc'])

constant_mortality = xr.DataArray(uf.mphi_f(-pred_prey_mat, 
                                            gridded_python['log10_pred_prey_ratio'],
                                            gridded_python['log_prey_pref'],
                                            gridded_python['metabolic_req_pred']),
                                  dims = ['size_class', 'sc'])

### Building metabolic requirements lookup table 
Metabolic requirements are estimated per size class

In [22]:
met_req_log10_size_bins = uf.expax_f(log10_size_bins_mat, 
                                     gridded_python['metabolic_req_pred'])

In [24]:
consume_pelagic = (pref_pelagic*
                   gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)
consume_benthos = (pref_benthos*
                   gridded_python['hr_volume_search']*
                   met_req_log10_size_bins)

### Creating a gridded time series of intercept of plankton size spectrum 
The size spectrum was estimated by the `GetPPIntSlope` function using GFDL-MOM6-COBALT2 outputs in script ([01_processing_dbpm_global_inputs](new_workflow/01_processing_dbpm_global_inputs.ipynb)).

In [26]:
ui0 = (10**int_phy_zoo).chunk({'lat': '50MB', 'lon': '50MB', 'time': '50MB'})

### Predator and detritivore biomass, plus detritus
Values are derived from plankton-zooplankton size spectrum, slope and size class.

In [27]:
# Calculating predator biomass
pred_init = ui0*(10**(slope_phy_zoo_mat*log10_size_bins_mat))
# Keep values outside the predator size classes, otherwise change to zero (0)
pred_init = xr.where(pred_init.size_class <= log10_ind_min_pred_size,
                     pred_init, 0)
# Copy first time step predator values as 'initialisation' timestep
pred_init = xr.concat([pred_init.isel(time = 0), pred_init], dim = 'time')
# Update time labels
pred_init['time'] = time_init

# Initialising detritivores
detri_init = xr.zeros_like(pred_init)

# Initialising detritus
detritus = xr.zeros_like(pred_init.isel(size_class = 0).drop_vars('size_class'))
# Assign detritus value to initial timestep
detritus = xr.where(detritus.time == detritus.time.min(), 
                    gridded_python['init_detritus'], detritus)

Setting initial biomass values for predator and detritivore from gridded parameters.

In [28]:
predators = xr.where((pred_init.time == pred_init.time.min()) & 
                      (pred_init.size_class >= log10_ind_min_pred_size),
                     init_pred, pred_init)
detritivores = xr.where((detri_init.time == detri_init.time.min()) & 
                        (detri_init.size_class >= log10_ind_min_detritivore_size),
                        init_detritivores, detri_init)

### Note
If no initial values for `predators` and `detritivores` are included in the gridded parameters, use the following code instead:  
  
```
predators = xr.where((pred_init.time == pred_init.time.min()) &
                       (pred_init.size_class >= log10_ind_min_pred_size) &
                       (pred_init.size_class < log10_size_bins[120]),
                       init_pred, pred_init)

detritivores = xr.where((detri_init.time == detri_init.time.min()) &
                         (detri_init.size_class >= log10_ind_min_detritivore_size) &
                         (detri_init.size_class < log10_size_bins[120]), 
                         init_detritivores, detri_init)
```

### Fishing mortality
Here `fishing_mort_pred` and `fishing_mort_det` are fixed catchability terms for `predators` and `detritivores`. Fishing mortality is estimated only for individuals within fished sizes (`ind_min_fish_pred` or `ind_min_fish_det`).

In [34]:
# Using fish_mort_pred as a base for fishing mortality for predators
fish_mort_pred = xr.DataArray(
    np.repeat(gridded_python['fish_mort_pred'], 
              np.prod(consume_pelagic.shape)).reshape(consume_pelagic.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_pelagic.lat, 'lon': consume_pelagic.lon,
             'size_class': consume_pelagic.size_class})

# Changing mortality to zero outside the predator size fished
fishing_mort_pred = xr.where((fish_mort_pred.size_class >= log10_ind_min_fish_pred) &
                          (fish_mort_pred.size_class < 
                           fish_mort_pred.size_class.max()),
                          fish_mort_pred, 0)

In [35]:
# Using fish_mort_det as a base for fishing mortality for detritivores
fish_mort_det = xr.DataArray(
    np.repeat(gridded_python['fish_mort_detritivore'], 
              np.prod(consume_benthos.shape)).reshape(consume_benthos.shape),
    dims = ['lat', 'lon', 'size_class'],
    coords = {'lat': consume_benthos.lat, 'lon': consume_benthos.lon,
             'size_class': consume_benthos.size_class})

# Changing mortality to zero outside the detritivore size fished
fishing_mort_det = xr.where((fish_mort_det.size_class >= log10_ind_min_fish_det) &
                          (fish_mort_det.size_class < 
                           fish_mort_det.size_class.max()),
                          fish_mort_det, 0)

### Temperature effect on pelagic and benthic organisms

In [33]:
pel_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_surf_temp+273)))

ben_tempeffect = np.exp(gridded_python['c1']-gridded_python['activation_energy']/
                        (gridded_python['boltzmann']*(sea_floor_temp+273)))

### Note
If `temp_effect` is not needed, then simply assign `1` to `pel_tempeffect` and `ben_tempeffect`:  
  
```
pel_tempeffect = 1
ben_tempeffect = 1
```

## Saving gridded inputs
We will now save all the gridded inputs needed to run the DBPM model.

In [34]:
log10_size_bins_mat.name = 'size_bins'
log10_size_bins_mat.to_zarr('outputs/log10_size_bins_matrix.zarr/',
                            consolidated = True, mode = 'w')

In [35]:
# Ensuring chunk is maximised
depth_out = depth.chunk({'lat': '50MB', 'lon': '50MB'})
depth_fn = os.path.join(out_folder, 
                        'gfdl-mom6-cobalt2_obsclim_deptho_15arcmin_fao-58_fixed.zarr/')

depth_out.to_zarr(depth_fn, consolidated = True, mode = 'w')

In [37]:
effort.name = 'effort'
effort.to_zarr(os.path.join(out_folder, 'effort_15arcmin_fao-58_monthly_1961_2010.zarr/'),
               consolidated = True, mode = 'w')

In [61]:
pref_benthos.name = 'pref_benthos'
pref_benthos.to_zarr(os.path.join(out_folder, 
                                  'pref-benthos_15arcmin_fao-58_fixed.zarr/'),
                     consolidated = True, mode = 'w')
pref_pelagic.name = 'pref_pelagic'
pref_pelagic.to_zarr(os.path.join(out_folder, 
                                  'pref-pelagic_15arcmin_fao-58_fixed.zarr/'), 
                     consolidated = True, mode = 'w')

In [39]:
constant_growth.name = 'constant_growth'
constant_growth.to_zarr(os.path.join(out_folder,
                                     'const-growth_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

constant_mortality.name = 'constant_mortality'
constant_mortality.to_zarr(os.path.join(out_folder,
                                     'const-mort_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [40]:
consume_pelagic.name = 'consume_pelagic'
consume_pelagic.to_zarr(os.path.join(out_folder,
                                     'consume-pelagic_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

consume_benthos.name = 'consume_benthos'
consume_benthos.to_zarr(os.path.join(out_folder,
                                     'consume-benthos_15arcmin_fao-58_fixed.zarr/'),
                        consolidated = True, mode = 'w')

In [59]:
predators.name = 'predators'
predators = predators.chunk({'time': 13, 'size_class': 181,
                             'lat': 23, 'lon': '50MB'})
predators.to_zarr(os.path.join(out_folder, 
                               'predators_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                  consolidated = True, mode = 'w')

detritivores.name = 'detritivores'
detritivores = detritivores.chunk({'time': 13, 'size_class': 181,
                                   'lat': 23, 'lon': '50MB'})
detritivores.to_zarr(os.path.join(out_folder, 
                                  'detritivores_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                     consolidated = True, mode = 'w')

detritus.name = 'detritus'
detritus = detritus.chunk({'time': '50MB', 'lat': 23,
                           'lon': '50MB'})
detritus.to_zarr(os.path.join(out_folder, 
                              'detritus_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                 consolidated = True, mode = 'w')

In [36]:
fishing_mort_pred.name = 'fish_mort_pred'
fishing_mort_pred.to_zarr(os.path.join(out_folder, 
                                       'fish-mort-pred_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')

fishing_mort_det.name = 'fish_mort_det'
fishing_mort_det.to_zarr(os.path.join(out_folder, 
                                      'fish-mort-det_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                         consolidated = True, mode = 'w')

In [67]:
pel_tempeffect.name = 'pel_temp_eff' 
pel_tempeffect = pel_tempeffect.chunk({'lat': 23})
pel_tempeffect.to_zarr(os.path.join(out_folder, 
                                       'pel-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')

ben_tempeffect.name = 'ben_temp_eff'
ben_tempeffect = ben_tempeffect.chunk({'lat': 23})
ben_tempeffect.to_zarr(os.path.join(out_folder, 
                                       'ben-temp-eff_15arcmin_fao-58_monthly_1961_2010.zarr/'),
                          consolidated = True, mode = 'w')